- https://nijianmo.github.io/amazon/index.html
- https://jmcauley.ucsd.edu/data/amazon/ 
- https://colab.research.google.com/drive/1Zv6MARGQcrBbLHyjPVVMZVnRWsRnVMpV

In [6]:
import pandas as pd


In [7]:
PRFX_PRP = 'prep_20210307B1'
PRFX = 'make_example_data_20210307A1'
HOME = "/data/git/product-category"
p_out = f'{HOME}/data/{PRFX}'
!mkdir -p {p_out}

In [8]:
sz = int(1e4)
DATA2USE = f'{HOME}/data/data_sample_{sz}__{PRFX_PRP}.csv'
df = pd.read_csv(DATA2USE, nrows=100)


In [12]:
df.to_csv(f'{p_out}/sample_data.csv', index=False)

# get data

    download from http://localhost:8080/notebooks/git/product-category/notebooks/secrets_get_amazon_data.ipynb

In [2]:
import pandas as pd
import gzip
import json
from collections import Counter

In [3]:
pdata = "/data/git/product-category/data/amazon_meta_data"
!ls -hl {pdata}

total 13G
-rw-rw-r-- 1 ubuntu ubuntu   30M Dec  2 00:07 meta_AMAZON_FASHION.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  9.6M Aug  7  2020 meta_All_Beauty.json.gz
-rw-rw-r-- 1 ubuntu ubuntu   57M Aug  7  2020 meta_Appliances.json.gz
-rw-rw-r-- 1 ubuntu ubuntu   57M Aug  7  2020 meta_Appliances.json.gz.1
-rw-rw-r-- 1 ubuntu ubuntu  125M Aug  7  2020 meta_Arts_Crafts_and_Sewing.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.5G Aug  7  2020 meta_Automotive.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.2G Aug  7  2020 meta_Books.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.2G Aug  7  2020 meta_Books.json.gz.1
-rw-rw-r-- 1 ubuntu ubuntu  153M Aug  7  2020 meta_CDs_and_Vinyl.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  333M Aug  7  2020 meta_Cell_Phones_and_Accessories.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.5G Dec  2 00:18 meta_Clothing_Shoes_and_Jewelry.json.gz
-rw-rw-r-- 1 ubuntu ubuntu   12M Aug  7  2020 meta_Digital_Music.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.2G Aug  7  2020 meta_Electronics.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  37

In [4]:
fnms = !ls {pdata}
fnms = [o for o in fnms if o.endswith('.json.gz')]
fnms

['meta_AMAZON_FASHION.json.gz',
 'meta_All_Beauty.json.gz',
 'meta_Appliances.json.gz',
 'meta_Arts_Crafts_and_Sewing.json.gz',
 'meta_Automotive.json.gz',
 'meta_Books.json.gz',
 'meta_CDs_and_Vinyl.json.gz',
 'meta_Cell_Phones_and_Accessories.json.gz',
 'meta_Clothing_Shoes_and_Jewelry.json.gz',
 'meta_Digital_Music.json.gz',
 'meta_Electronics.json.gz',
 'meta_Gift_Cards.json.gz',
 'meta_Grocery_and_Gourmet_Food.json.gz',
 'meta_Home_and_Kitchen.json.gz',
 'meta_Industrial_and_Scientific.json.gz',
 'meta_Kindle_Store.json.gz',
 'meta_Luxury_Beauty.json.gz',
 'meta_Magazine_Subscriptions.json.gz',
 'meta_Movies_and_TV.json.gz',
 'meta_Musical_Instruments.json.gz',
 'meta_Office_Products.json.gz',
 'meta_Patio_Lawn_and_Garden.json.gz',
 'meta_Pet_Supplies.json.gz',
 'meta_Prime_Pantry.json.gz',
 'meta_Software.json.gz',
 'meta_Sports_and_Outdoors.json.gz',
 'meta_Tools_and_Home_Improvement.json.gz',
 'meta_Toys_and_Games.json.gz',
 'meta_Video_Games.json.gz']

In [5]:
domains = [o.strip('meta_').strip('.json.gz') for o in fnms]
dmn2fnm = dict(zip(domains, fnms))
dmn2fnm

{'AMAZON_FASHION': 'meta_AMAZON_FASHION.json.gz',
 'All_Beauty': 'meta_All_Beauty.json.gz',
 'Appliance': 'meta_Appliances.json.gz',
 'Arts_Crafts_and_Sewi': 'meta_Arts_Crafts_and_Sewing.json.gz',
 'Automotive': 'meta_Automotive.json.gz',
 'Book': 'meta_Books.json.gz',
 'CDs_and_Vinyl': 'meta_CDs_and_Vinyl.json.gz',
 'Cell_Phones_and_Accessorie': 'meta_Cell_Phones_and_Accessories.json.gz',
 'Clothing_Shoes_and_Jewelry': 'meta_Clothing_Shoes_and_Jewelry.json.gz',
 'Digital_Music': 'meta_Digital_Music.json.gz',
 'Electronic': 'meta_Electronics.json.gz',
 'Gift_Card': 'meta_Gift_Cards.json.gz',
 'Grocery_and_Gourmet_Food': 'meta_Grocery_and_Gourmet_Food.json.gz',
 'Home_and_Kitche': 'meta_Home_and_Kitchen.json.gz',
 'Industrial_and_Scientific': 'meta_Industrial_and_Scientific.json.gz',
 'Kindle_Store': 'meta_Kindle_Store.json.gz',
 'Luxury_Beauty': 'meta_Luxury_Beauty.json.gz',
 'Magazine_Subscripti': 'meta_Magazine_Subscriptions.json.gz',
 'Movies_and_TV': 'meta_Movies_and_TV.json.gz',
 

In [6]:
KEYS2USE = set(['category', 'title',])

In [7]:
# https://colab.research.google.com/drive/1Zv6MARGQcrBbLHyjPVVMZVnRWsRnVMpV#scrollTo=7igYuRaV4bF7
    
### load the meta data
def get_meta_data(domain, nrows=None):
    keys2use = KEYS2USE
    fnm = dmn2fnm[domain]
    data = []
    with gzip.open(f'{pdata}/{fnm}') as f:
        for i,l in enumerate(f):
            dat = json.loads(l.strip())
            dat = {k:v for k,v in dat.items() if k in keys2use}
            data.append(dat)
            if nrows and i>nrows: break

    # total length of list, this number equals total number of products
#     print("len(data)", len(data))

    # convert list into pandas dataframe
    df = pd.DataFrame.from_dict(data)
#     print("len(df)", len(df))

    ### remove rows with unformatted title (i.e. some 'title' may still contain html style content)
    df3 = df.dropna()
    df4 = df3[df3.title.str.contains('getTime')] # unformatted rows
    df5 = df3[~df3.title.str.contains('getTime')] # filter those unformatted rows
#     print("len(df4)", len(df4))
#     print("len(df5)", len(df5))

    return df5

In [8]:
df = get_meta_data(domains[3], nrows=10000)
df.head(3)

,category,title
0,"[Arts, Crafts & Sewing, Sewing, Trim & Embelli...",You Son of a Bitch! 1987 Embroidered Patch
1,"[Arts, Crafts & Sewing, Crafting, Paper & Pape...",Origami Stars Papers Package 1H (5 packs)
2,"[Arts, Crafts & Sewing, Painting, Drawing & Ar...",Yi De Ge Chinese Calligraphy Sumi Drawing Blac...


# preprocess data

## concate

In [9]:
dfs = []
for dmn in domains:
    print(dmn)
    df = get_meta_data(dmn, nrows=None)
    if set(df.columns)!= KEYS2USE: continue
    df = df[df.category.apply(len)>0]
    if len(df)==0: continue
    print(df.shape)
    display(df.sample(2))
    dfs.append(df)

AMAZON_FASHION
All_Beauty
Appliance
(29639, 2)


,category,title
27618,"[Appliances, Parts &amp; Accessories]",Minn Kota Riptide Ulterra 80/IP w/i-Pilot - No...
19299,"[Appliances, Parts & Accessories, Range Parts ...",Frigidaire 316218401 Control Knob Range/Stove/...


Arts_Crafts_and_Sewi
(284889, 2)


,category,title
54809,"[Arts, Crafts & Sewing, Sewing, Trim & Embelli...",City Of Los Angeles Patch 3&quot;
206018,"[Arts, Crafts & Sewing, Sewing, Trim & Embelli...","Cotton Linen Blend Fabric Ribbon, 5/8-inch, 25..."


Automotive
(897587, 2)


...

,category,title
18539,"[Tools & Home Improvement, Power & Hand Tools,...",Milwaukee 48-52-5010 4-Inch Stringer Bead Wheel
504525,"[Tools & Home Improvement, Electrical]",Sunward Natural Rock Crystal Quartz Hexagonal ...


Toys_and_Game
(571640, 2)


,category,title
264601,"[Toys & Games, Party Supplies, Party Packs]",SpongeBob SquarePants Theme Birthday Party or ...
449082,"[Toys & Games, Dress Up & Pretend Play, Access...",Aviator Black Bomber Hat &amp; Goggles - One Size


Video_Game
(82394, 2)


,category,title
45129,"[Video Games, Retro Gaming & Microconsoles, Pl...",Naruto Shippuden Ultimate Ninja 5 Ps2 PAL
19004,"[Video Games, Retro Gaming & Microconsoles, Ga...",Pokemon GB Color Carry Case-GB4 - Game Boy Color


In [10]:
df = pd.concat(dfs)
df.shape

(13706615, 2)

In [11]:
df.sample(3)

,category,title
977765,"[Home & Kitchen, Kitchen & Dining, Dining & En...","BUNN Glass Coffee Pot Decanter/Carafe, Orange,..."
270428,"[Tools & Home Improvement, Power & Hand Tools,...",PT 37875 Pipe Dies 3/4&quot; Manual Pipe threa...
17134,"[Grocery & Gourmet Food, Candy & Chocolate, Ha...","Rockit Energy Pops, 80-Count Display Box"


## cleanup

### category

In [12]:
%%time
def try2eval(x):
    try:
        x = eval(x)
        x = (o.replace('|','') for o in x)
        return '|'.join(x)
    except SyntaxError:
        return ""

df.category = df.category.astype(str)
df.category = df.category.apply(try2eval)
print("df.shape:", df.shape)

df.shape: (13706615, 2)
CPU times: user 3min 31s, sys: 4.81 s, total: 3min 36s
Wall time: 3min 36s


### text

In [13]:
import re

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [14]:
%%time
df.title = df.title.apply(cleanhtml)

CPU times: user 34.9 s, sys: 200 ms, total: 35.1 s
Wall time: 35.1 s


# train val split

In [15]:
import numpy as np

np.random.seed(101)
df['is_validation'] = np.random.choice(2, size=len(df), p=[0.85,0.15])

In [16]:
df.is_validation.value_counts()

0    11649808
1     2056807
Name: is_validation, dtype: int64

In [17]:
df.sample(3)

,category,title,is_validation
966930,Home & Kitchen|Furniture,"1,000-lb. Capacity Heavy-Duty Portable Chair",0
412242,Automotive|Replacement Parts|Brake System|Rotors,DuraGo BP691 MS Front Semi-Metallic Brake Pad,0
508706,Books|Literature &amp; Fiction|Short Stories &...,Sevenfold,0


# save

In [18]:
%%time
df.to_csv(f'../data/data__{prfx}.csv', index=False)

CPU times: user 1min 32s, sys: 4.58 s, total: 1min 37s
Wall time: 1min 38s


In [19]:
ls -hl ../data/data__{prfx}.csv

-rw-rw-r-- 1 ubuntu ubuntu 2.1G Mar  7 16:48 ../data/data__prep_20210307B1.csv


## make a sample

In [20]:
import numpy as np
np.random.seed(101)
df_sample = df.sample(10000)

In [21]:
df_sample.to_csv(f'../data/data_sample__{prfx}.csv', index=False)

## make more samples

In [3]:
%%time
df = pd.read_csv(f'../data/data__{prfx}.csv')

CPU times: user 22.2 s, sys: 3.06 s, total: 25.2 s
Wall time: 33.2 s


In [4]:
len(df)

13706615

In [5]:
for sz in (int(o) for o in (1e4, 1e5, 5e5, 1e6, 5e6)):
    df_sample = df.sample(sz)
    df_sample.to_csv(f'../data/data_sample_{sz}__{prfx}.csv', index=False)